<a href="https://colab.research.google.com/github/jforbes02/Parallel_Computing_Project_Repo/blob/main/MatrixMulitiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#install CUDA C++ plugin for Colab

!pip install nvcc4jupyter
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [6]:
#Serial code
import random
from typing import List

m = int(input("Enter the number of rows: "))
n = int(input("Enter the number of columns: "))
a = int(input("Enter the number of rows for the second matrix: "))
b = int(input("Enter the number of columns for the second matrix:"))
matrix1 = []
matrix2 = []
for i in range(m):
  row = []
  for j in range(n):
        row.append(random.randint(0, 10))
  matrix1.append(row)

for i in range(a):
  row = []
  for j in range(b):
    row.append(random.randint(0, 10))
  matrix2.append(row)

def multiply(x, y) -> List:
  rows_x = len(x)
  cols_x = len(x[0])
  rows_y = len(y)
  cols_y = len(y[0])
  if cols_x != rows_y:
    raise ValueError("Cannot multiply matrices with these dimensions.")

  #What is being returned
  result = [[0 for _ in range(cols_y)] for _ in range(rows_x)]


  for i in range(rows_x):
    for j in range(cols_y):
      for k in range(cols_x):
        result[i][j] += x[i][k] * y[k][j]
  return result
print(matrix1)
print(matrix2)
Ans = multiply(matrix1, matrix2)
print(Ans)


Enter the number of rows: 2
Enter the number of columns: 2
Enter the number of rows for the second matrix: 2
Enter the number of columns for the second matrix:2
[[2, 5], [10, 2]]
[[6, 4], [0, 8]]
[[12, 48], [60, 56]]


In [7]:
# Detect selected GPU and its NVIDIA architecture:
import subprocess
gpu_info = subprocess.getoutput("nvidia-smi --query-gpu=name,compute_cap --format=csv,noheader,nounits")
if "not found" in gpu_info.lower(): raise RuntimeError("Error: No GPU found. Please select a GPU runtime environment.")

gpu_name, compute_cap = map(str.strip, gpu_info.split(","))
gpu_arch = f"sm_{compute_cap.replace('.','')}"

print(f"{'GPU Name':<15}: {gpu_name}")
print(f"{'GPU Arch.':<15}: {gpu_arch}")


GPU Name       : NVIDIA A100-SXM4-40GB
GPU Arch.      : sm_80


In [29]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <cstdlib>

__global__ void matrix_mul(int *a, int *b, int *c, int rows_1, int cols_1, int rows_2, int cols_2){
    int col = threadIdx.x + blockIdx.x * blockDim.x;
    int row = threadIdx.y + blockIdx.y * blockDim.y;

    if (row < rows_1 && col < cols_2){
      int sum = 0;
      for(int i = 0; i < cols_1; i++){
        sum += a[row * cols_1 + i] * b[i * cols_2 + col];
      }
      c[row * cols_2 + col] = sum;
  }
}

void init_matrix(int *m, int rows, int cols){
  int N = rows * cols;
  for (int i = 0; i < N; i++){
      m[i]  = rand() % 10;
    }
}

void print_matrix(int *m, int rows, int cols){
  for (int i = 0; i < rows; i++){
    for (int j = 0; j < cols; j++){
      printf("%d ", m[i * cols + j]);
    }
    printf("\n");
  }
}

int main(){
    // Get sizes of the matrices
  int x, y, a, b; //x and a are rows, y and b columns
  printf("Enter an integer for the row of matrix 1");
  scanf("%d", &x);
  printf("Enter an integer for the column of matrix 1");
  scanf("%d", &y);
  printf("Enter an integer for the row of matrix 2");
  scanf("%d", &a);
  printf("Enter an integer for the column of matrix 2");
  scanf("%d", &b);
    // must be multiplicable
  if (y != a){
      printf("The matrices cannot be multiplied");
      return 0;
  }
    //sizes of matrix dimensions
  size_t m1Size = x * y * sizeof(int);
  size_t m2Size = a * b * sizeof(int);
  size_t outputSize = x * b * sizeof(int);

    // Allocate memory for matrices
  int *matrix1 = (int *)malloc(m1Size);
  int *matrix2 = (int *)malloc(m2Size);
  int *output_matrix = (int *)malloc(outputSize);

    //initalize matrix w random num
  init_matrix(matrix1, x, y);
  init_matrix(matrix2, a, b);
    //original matrices
  print_matrix(matrix1, x, y);
  print_matrix(matrix2, a, b);

  //Allocate matrix memory on GPU
  int *d_matrix1, *d_matrix2, *d_output_matrix;
  cudaMalloc((void**)&d_matrix1, m1Size);
  cudaMalloc((void**)&d_matrix2, m2Size);
  cudaMalloc((void**)&d_output_matrix, outputSize);
  // copy data to gpu
  cudaMemcpy(d_matrix1, matrix1, m1Size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_matrix2, matrix2, m2Size, cudaMemcpyHostToDevice);

  int threads = 16;
  int blocks = (x * b + threads - 1) / threads;

  //kernal launch
  dim3 dimBlocks(blocks, blocks);
  dim3 dimThreads(threads, threads);

  matrix_mul<<<dimBlocks, dimThreads>>>(matrix1, matrix2, output_matrix, x, y, a, b);
  cudaDeviceSynchronize();

  // device to the host
  cudaMemcpy(output_matrix, d_output_matrix, outputSize, cudaMemcpyDeviceToHost);

  printf("Output Matrix:\n");
  print_matrix(output_matrix, x, b);

  cudaFree(matrix1);
  cudaFree(matrix2);
  cudaFree(output_matrix);
  free(matrix1);
  free(matrix2);
  free(output_matrix);
  return 0;
}


Enter an integer for the row of matrix 1Enter an integer for the column of matrix 1Enter an integer for the row of matrix 2Enter an integer for the column of matrix 2Output Matrix:



In [9]:
# 2D matrix multiplication (mxn * axb)
# User Input Row x Columnn num ex (2, 4) in total 4 inputs
# Randomly Generated numbers inside of the matrix
# Take into account when columns/rows are not divisible by # of cores
# Content of the input and output should be printed